# Phase 3.2: Managing Model Versions and Stages

This comprehensive notebook demonstrates:
1. **Creating Multiple Versions** - Train and register multiple model versions
2. **Model Stages** - Understanding None, Staging, Production, Archived
3. **Stage Transitions** - Moving models between stages
4. **Archiving Models** - Retiring old model versions

## What are Model Stages?

MLflow Model Registry uses **stages** to track where a model is in its lifecycle:

| Stage | Purpose | When to Use |
|-------|---------|-------------|
| **None** | Default | Just registered, not yet evaluated |
| **Staging** | Testing | Being validated before production |
| **Production** | Live | Serving real traffic |
| **Archived** | Retired | No longer in use, kept for history |

## Typical Workflow

```
None -> Staging -> Production -> Archived
       (test)     (deploy)      (retire)
```

## Learning Goals
- Create multiple model versions
- Understand the stage lifecycle
- Transition models between stages
- Archive old model versions

## Step 1: Import Libraries

In [ ]:
# MLflow imports
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

# sklearn imports
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Data handling
import pandas as pd
import time
import os

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")
print("Ready to learn about model stages!")

## Step 2: Connect to MLflow

In [ ]:
# Get MLflow tracking server URL
TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI", "http://localhost:5000")

# Connect to MLflow
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment("phase3-model-stages")

# Model name for this demo
MODEL_NAME = "iris-classifier-staged"

# Create client for registry operations
client = MlflowClient()

print(f"Connected to MLflow at: {TRACKING_URI}")
print(f"Experiment: phase3-model-stages")
print(f"Model name: {MODEL_NAME}")

## Step 3: Prepare Data

In [ ]:
# Load and split data
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Data loaded and split!")
print(f"Training: {len(X_train)} samples")
print(f"Testing: {len(X_test)} samples")

## Step 4: Clean Up Existing Model (For Demo)

To ensure a clean demonstration, we'll delete any existing model with the same name.

In [ ]:
# Delete existing model if it exists (for clean demo)
try:
    client.delete_registered_model(MODEL_NAME)
    print(f"Deleted existing model: {MODEL_NAME}")
except:
    print(f"No existing model named '{MODEL_NAME}' to delete.")

## Step 5: Create Multiple Model Versions

Let's train 3 different models with different hyperparameters and register each as a new version.

In [ ]:
print("="*60)
print("[1] Creating Multiple Model Versions")
print("="*60)

# Define different configurations to try
# Each will become a different version in the registry
configs = [
    {"n_estimators": 50, "max_depth": 3},   # Version 1: Small model
    {"n_estimators": 100, "max_depth": 5},  # Version 2: Medium model
    {"n_estimators": 150, "max_depth": 10}, # Version 3: Large model
]

# Store version info for later use
versions_info = []

print("\nTraining and registering models:")
print("-" * 50)

for i, config in enumerate(configs, 1):
    with mlflow.start_run(run_name=f"version-{i}"):
        # Train model with this configuration
        model = RandomForestClassifier(**config, random_state=42)
        model.fit(X_train, y_train)
        
        # Evaluate
        accuracy = accuracy_score(y_test, model.predict(X_test))
        
        # Log parameters and metrics
        mlflow.log_params(config)
        mlflow.log_metric("accuracy", accuracy)
        
        # Log and register model (creates new version each time)
        mlflow.sklearn.log_model(
            model,
            "model",
            registered_model_name=MODEL_NAME  # Auto-registers!
        )
        
        # Store info
        versions_info.append({
            "version": i,
            "config": config,
            "accuracy": accuracy
        })
        
        print(f"  Version {i}: n_estimators={config['n_estimators']:3d}, "
              f"max_depth={config['max_depth']:2d} | accuracy={accuracy:.4f}")
        
        # Small delay to ensure registry updates
        time.sleep(1)

print("-" * 50)

# Find best model
best = max(versions_info, key=lambda x: x["accuracy"])
print(f"\nBest model: Version {best['version']} (accuracy: {best['accuracy']:.4f})")

## Step 6: Understand Model Stages

All new models start in the **None** stage. Let's see the current state of our models.

In [ ]:
print("\n" + "="*60)
print("[2] Current Model Stages")
print("="*60)

print("\nAll models start in 'None' stage:")
print("-" * 40)

# List all versions and their stages
for mv in client.search_model_versions(f"name='{MODEL_NAME}'"):
    print(f"  Version {mv.version}: Stage = {mv.current_stage}")

print("\n" + "="*60)
print("Available Stages")
print("="*60)
print("""
  None        -> Just registered, not evaluated yet
      |
      v
  Staging     -> Being tested/validated
      |
      v
  Production  -> Serving live traffic
      |
      v
  Archived    -> Retired, kept for history
""")

## Step 7: Transition Models to Stages

Now let's move our models to appropriate stages based on their performance.

In [ ]:
print("\n" + "="*60)
print("[3] Transitioning Models to Stages")
print("="*60)

# Transition models based on accuracy
for info in versions_info:
    version = str(info["version"])
    accuracy = info["accuracy"]
    
    if info["version"] == best["version"]:
        # Best model goes to Production
        client.transition_model_version_stage(
            name=MODEL_NAME,
            version=version,
            stage="Production",
            archive_existing_versions=False  # Don't archive other Production versions
        )
        print(f"\n  Version {version} -> Production (best accuracy: {accuracy:.4f})")
        
    elif accuracy > 0.9:
        # Good models go to Staging
        client.transition_model_version_stage(
            name=MODEL_NAME,
            version=version,
            stage="Staging",
            archive_existing_versions=False
        )
        print(f"  Version {version} -> Staging (good accuracy: {accuracy:.4f})")
        
    else:
        # Keep in None stage
        print(f"  Version {version} -> None (needs improvement: {accuracy:.4f})")

## Step 8: View Current Registry State

In [ ]:
print("\n" + "="*60)
print("[4] Current Registry State")
print("="*60)

for mv in client.search_model_versions(f"name='{MODEL_NAME}'"):
    # Get the run to show metrics
    run = client.get_run(mv.run_id)
    accuracy = run.data.metrics.get("accuracy", "N/A")
    
    print(f"\n  Version {mv.version}:")
    print(f"    Stage: {mv.current_stage}")
    print(f"    Status: {mv.status}")
    print(f"    Accuracy: {accuracy:.4f}" if isinstance(accuracy, float) else f"    Accuracy: {accuracy}")
    print(f"    Run ID: {mv.run_id[:8]}...")

## Step 9: Archive Old Versions

When a model version is no longer needed, we can archive it to keep the registry clean while preserving history.

In [ ]:
print("\n" + "="*60)
print("[5] Archiving Old Versions")
print("="*60)

# Archive version 1 (the oldest/smallest model)
client.transition_model_version_stage(
    name=MODEL_NAME,
    version="1",
    stage="Archived",
    archive_existing_versions=False
)

print("\n  Version 1 -> Archived")
print("  (This model is now retired but still accessible for reference)")

## Step 10: Final Registry State

In [ ]:
print("\n" + "="*60)
print("Final Registry State")
print("="*60)

print(f"\nModel: {MODEL_NAME}")
print("-" * 40)

for mv in client.search_model_versions(f"name='{MODEL_NAME}'"):
    # Get metrics
    run = client.get_run(mv.run_id)
    accuracy = run.data.metrics.get("accuracy", 0)
    
    # Format stage nicely
    stage_display = mv.current_stage.ljust(12)
    
    print(f"  Version {mv.version}: {stage_display} (accuracy: {accuracy:.4f})")

print("\n" + "-" * 40)
print("Legend:")
print("  Production = Currently serving")
print("  Staging    = Being tested")
print("  None       = Newly registered")
print("  Archived   = Retired")

## Summary: Model Stages

### Stage Transition Function

```python
client.transition_model_version_stage(
    name="model-name",           # Registered model name
    version="1",                 # Version number (as string)
    stage="Production",          # Target stage
    archive_existing_versions=False  # Whether to auto-archive others
)
```

### Key Points

1. **Stages are for lifecycle management**, not performance comparison
2. **Only one version should be in Production** at a time (typically)
3. **Use archive_existing_versions=True** to auto-archive when promoting to Production
4. **Archived models are still accessible** - just marked as retired

### Common Workflow

```
1. Train new model        -> Version N created (None stage)
2. Validate model         -> Promote to Staging
3. Test in staging        -> Run integration tests
4. Deploy to production   -> Promote to Production
5. New model ready        -> Archive old production
```

In [ ]:
print("="*60)
print("Model Stages Tutorial Complete!")
print("="*60)
print(f"\nView at: {TRACKING_URI}/#/models/{MODEL_NAME}")
print("\nWhat you learned:")
print("  1. How to create multiple model versions")
print("  2. What the 4 model stages mean (None, Staging, Production, Archived)")
print("  3. How to transition models between stages")
print("  4. How to archive old model versions")
print("  5. Best practices for model lifecycle management")